# MaxQuant (MQ) Output-Files

Files compared:
1. `Summary.txt`
2. `mqpar.xml`
3. `peptides.txt`
4. `proteins.txt`

There is are many files more, where several files seem to be available in several times in different formats.

In [ ]:
import os
import logging
from pathlib import Path

import pandas as pd
import ipywidgets as widgets

from vaep.io import search_files, search_subfolders, PathsList

import src
import src.file_utils as file_io
from src.file_utils import check_for_key
from src.file_utils import process_files
from src.file_utils import load_summary, load_mqpar_xml

##################
##### CONFIG #####
##################
from config import FOLDER_RAW_DATA, FOLDER_PROCESSED
from config import FOLDER_KEY  # defines how filenames are parsed for use as indices

from config import FOLDER_DATA # project folder for storing the data
print(f"Search Raw-Files on path: {FOLDER_RAW_DATA}")

In [ ]:
from datetime import datetime

#Delete Jupyter notebook root logger handler
logger = logging.getLogger()
logger.handlers = []

# logger = logging.getLogger(mq_output.folder.stem)
logger = logging.getLogger('vaep')
logger.setLevel(logging.INFO)

# c_handler = logging.StreamHandler()
# c_handler.setLevel(logging.INFO)

date_log_file = "{:%y%m%d_%H%M}".format(datetime.now())
f_handler = logging.FileHandler(f"log_01_maxquant_file_processing_{date_log_file}.txt")
f_handler.setLevel(logging.INFO)

c_format = logging.Formatter(
    f'%(name)s - %(levelname)-8s %(message)s ')

# c_handler.setFormatter(c_format)
f_handler.setFormatter(c_format)

logger.handlers = []  #remove any handler in case you reexecute the cell
# logger.addHandler(c_handler)
logger.addHandler(f_handler)

In [ ]:
folders= search_subfolders(path=FOLDER_RAW_DATA, depth=1, exclude_root=True)
folders[:10]

Results will be saved in a subfolder under `vaep/project/data` using the name of the specified input-folder per default. Change to your liking:

In [ ]:
all_files = search_files(path=FOLDER_RAW_DATA, query='')

In [ ]:
all_files.folder

In [ ]:
all_files.files[:10]

> Go to the block you are interested in!

## MQ Summary files

In [ ]:
paths_summaries = PathsList([file for file in all_files.files if 'summary.txt' in file], folder=all_files.folder)
w_file = widgets.Dropdown(options=paths_summaries.files, description='View files')
w_file

### File Handler

In [ ]:
load_summary??

### Summaries

In [ ]:
if paths_summaries.files:
    df, names, failed = process_files(handler_fct=load_summary, filepaths=paths_summaries.files, key=FOLDER_KEY, relative_to=paths_summaries.folder)
    df.columns = names
    print(f"Number of failed reads: {len(failed)}")
    display(df)

In [ ]:
if paths_summaries.files:
    df.to_csv(os.path.join(FOLDER_PROCESSED, 'all_summary_txt.csv'))
    df.to_pickle(os.path.join(FOLDER_PROCESSED, 'all_summary_txt.pkl'))

- SIL - MS2 based on precursor which was a set of peaks
- PEAK - MS2 scan based on a single peak on precursor spectrum
- ISO - isotopic pattern detection


In [ ]:
if paths_summaries.files:
    MS_spectra = df.loc[['MS', 'MS/MS Identified']].T.astype('int64')
    mask  = MS_spectra['MS/MS Identified'] > 0
    display(MS_spectra.loc[mask].describe())
    MS_spectra.to_csv(os.path.join(FOLDER_PROCESSED, 'overview_stats.csv'))

## MaxQuant Parameter File

In [ ]:
paths_parameters = PathsList(files=[file for file in all_files.files if '.xml' in file], folder=all_files.folder)
w_file = widgets.Dropdown(options=paths_parameters.files, description='Select a file')
w_file

### Parameter Files

In [ ]:
load_mqpar_xml??

In [ ]:
fname_mqpar_xml = os.path.join(FOLDER_PROCESSED, 'peptide_intensities.{}')

if paths_parameters.files:
    df, col_names, failed = process_files(handler_fct=load_mqpar_xml, filepaths=paths_parameters.files, key=FOLDER_KEY, relative_to=paths_parameters.folder) 
    df.columns = col_names
    print(f"Number of failed reads: {len(failed)}")
    pd.set_option('max_rows', 160)
    display(df)
    df.to_pickle(fname_mqpar_xml.format("pkl"))

In [ ]:
del df

## Peptides

In [ ]:
paths_peptides = PathsList(files = [file for file in all_files.files if 'peptides.txt' in file], folder=all_files.folder)
# paths_peptides

In [ ]:
import random
from vaep.io.mq import MaxQuantOutputDynamic
pd.set_option('max_columns', 60)

mq_output = MaxQuantOutputDynamic(
    folder=folders[random.randint(0, len(paths_peptides.files)-1)])
mq_output.peptides

In [ ]:
mq_output.peptides.Intensity # as is in peptides.txt, comma seperated thousands

### Create peptide intensity dumps for each MQ outputfolder

All folders are stored in a list

In [ ]:
folders[:10]

Check if the output folder contains already parsed files

In [ ]:
import json

import config

with open(config.FN_FASTA_DB) as f:
    data_fasta = json.load(f)
print(f'Number of proteins in fasta file DB: {len(data_fasta)}')

In [ ]:
# root_logger = logging.getLogger()
# root_logger.handlers = []
# root_logger.handlers

In [ ]:
%%time
from pathlib import Path
from vaep.io.mq import ExtractFromPeptidesTxt

FOLDER_PROCESSED = Path(FOLDER_PROCESSED)
set_previously_loaded =  {folder.name for folder in FOLDER_PROCESSED.iterdir()}

FORCE = True

for folder in folders:
    if folder.name in set_previously_loaded and not FORCE:
        pass
    else:
        print(f"Process: {folder.name}")
        mq_output = MaxQuantOutputDynamic(folder)
        peptide_extractor = ExtractFromPeptidesTxt(
            out_folder=FOLDER_PROCESSED, mq_output_object=mq_output, fasta_db=data_fasta)
        completeness_per_gene = peptide_extractor()

## Theoretial Peptides from used fasta-file

> `01_explore_FASTA.ipynb` (formely `01_FASTA_tryptic_digest.ipynb`)